In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Restrict GPU to 1 device if needed
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Optional: helps debug CUDA errors clearly
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [3]:
!pip install torch==2.7.1
!pip install transformers==4.54.0
!pip install unsloth
!pip install trl==0.8.6
!pip install -U accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 26.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 27.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 19.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 7.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 8.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 6.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━

In [4]:
from unsloth import FastLanguageModel
import torch


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-07-27 21:09:51.142466: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753650591.495806      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753650591.600899      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
model_dir = "/kaggle/input/deepseekmodel" 
model, tokenizer = FastLanguageModel.from_pretrained(model_dir)
FastLanguageModel.use_fused_cross_entropy = False  # Disable fused loss to avoid errors
model.eval()  # Set to eval mode for inference


==((====))==  Unsloth 2025.7.8: Fast Llama patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

Unsloth 2025.7.8 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (

In [8]:
prompt = (
    "### Instruction:\n"
    "I booked my train from Hyderabad to Kerala and the train was cancelled without intimation. "
    "I had an interview, but because of the train cancellation I did not go. "
    "Can I sue the railways for this?\n\n"
    "### Response:"
)

inputs = tokenizer(prompt, return_tensors="pt").to(torch.cuda.current_device())

with torch.no_grad():
    output_ids = model.generate(
        inputs["input_ids"],
        max_length=512,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


### Instruction:
I booked my train from Hyderabad to Kerala and the train was cancelled without intimation. I had an interview, but because of the train cancellation I did not go. Can I sue the railways for this?

### Response: 
Yes, you can sue the railways for the cancellation of the train without intimation, which caused you to miss the interview. This falls under the category of 'negligence' by the railway authorities. You may be entitled to compensation for the loss caused due to this negligence.  The railway authorities are legally bound to provide a proper explanation for the cancellation and may be required to compensate you for the loss.  You should consult a lawyer to understand your rights and the appropriate legal action to take.  The railway authorities may be held liable for the cancellation of the train without intimation, which resulted in you missing the interview.  You can file a complaint with the Railways Ombudsman or approach the Consumer Court to seek compensation

In [7]:
prompt = (
    "### Instruction:\n"
    "Under which laws in India can I sue the railways for train cancellation without intimation and what is the complaint procedure? "
    "Also, whom do I sue and which court has jurisdiction?\n\n"
    "### Response:"
)


inputs = tokenizer(prompt, return_tensors="pt").to(torch.cuda.current_device())

with torch.no_grad():
    output_ids = model.generate(
        inputs["input_ids"],
        max_length=256,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


### Instruction:
Under which laws in India can I sue the railways for train cancellation without intimation and what is the complaint procedure? Also, whom do I sue and which court has jurisdiction?

### Response: 
Under the Indian Railways Act, 1989, I can sue the railway company in the Consumer Disputes Redressal Commission. The complaint can be filed in the district consumer court. The railway company is the defendant. The jurisdiction is the consumer court. The complaint procedure involves filing a written complaint with the district consumer court. The court will investigate the matter and decide whether the railway company is at fault. If the court finds the railway company at fault, it may order compensation or other relief. The railway company may appeal the decision. The process is similar to filing a consumer complaint. The railway company is the defendant. The jurisdiction is the consumer court. The complaint procedure involves filing a written complaint with the district co

In [13]:
prompt = (
    "### Instruction:\n"
    "Under which laws in India can I sue the railways for train cancellation without intimation and what is the complaint procedure? "
    "Also, whom do I sue and which court has jurisdiction?\n\n"
    "### Response:"
)


inputs = tokenizer(prompt, return_tensors="pt").to(torch.cuda.current_device())

with torch.no_grad():
    output_ids = model.generate(
        inputs["input_ids"],
        max_length=256,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


### Instruction:
Under which laws in India can I sue the railways for train cancellation without intimation and what is the complaint procedure? Also, whom do I sue and which court has jurisdiction?

### Response: 
Under the Indian Railways Act, 1989, I can sue the railway company in the Consumer Disputes Redressal Commission. The complaint can be filed in the district consumer court. The railway company is the defendant. The jurisdiction is the consumer court. The complaint procedure involves filing a written complaint with the district consumer court. The court will investigate the matter and decide whether the railway company is at fault. If the court finds the railway company at fault, it may order compensation or other relief. The railway company may appeal the decision. The process is similar to filing a consumer complaint. The railway company is the defendant. The jurisdiction is the consumer court. The complaint procedure involves filing a written complaint with the district co

In [14]:
prompt = (
    "### Instruction:\n"
    "Critically analyze whether the doctrine of basic structure, as established by the Indian Supreme Court, acts as a check on parliamentary sovereignty or is itself an undemocratic overreach by the judiciary. Cite comparative constitutional examples where relevant.\n\n"
    "### Response:"
)


inputs = tokenizer(prompt, return_tensors="pt").to(torch.cuda.current_device())

with torch.no_grad():
    output_ids = model.generate(
        inputs["input_ids"],
        max_length=512,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


### Instruction:
Critically analyze whether the doctrine of basic structure, as established by the Indian Supreme Court, acts as a check on parliamentary sovereignty or is itself an undemocratic overreach by the judiciary. Cite comparative constitutional examples where relevant.

### Response: 
The doctrine of basic structure is seen as a necessary check on the legislature's ability to amend the Constitution, ensuring that fundamental rights and essential features of the Constitution remain intact. It is not an undemocratic overreach but a necessary safeguard for the democratic governance of India. This is similar to the concept of constitutional amendment limits in other countries, such as the United States, where amendments are subject to certain restrictions. However, the application of the doctrine in India has sometimes been criticized for being too broad and not providing adequate guidance to the judiciary on when to strike down legislative actions. This is a complex issue that r

In [16]:
prompt = (
    "### Instruction:\n"
    "If a government invokes national security to limit free speech, how should constitutional courts distinguish legitimate restrictions from concealed censorship? Provide a reasoned framework referencing jurisprudence from India, the US, and Europe.\n\n"
    "### Response:"
)
inputs = tokenizer(prompt, return_tensors="pt").to(torch.cuda.current_device())

with torch.no_grad():
    output_ids = model.generate(
        inputs["input_ids"],
        max_length=512,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


### Instruction:
If a government invokes national security to limit free speech, how should constitutional courts distinguish legitimate restrictions from concealed censorship? Provide a reasoned framework referencing jurisprudence from India, the US, and Europe.

### Response: 
The court should apply a proportionality test, ensuring that the restriction is necessary, effective, and proportionate to the threat. It should also consider transparency and accountability mechanisms. The Indian Supreme Court's cases on sedition and free speech provide a framework for this analysis. Similarly, the US Supreme Court's cases on the First Amendment and European Court of Human Rights' cases on freedom of expression offer guidance on balancing national security with democratic values.


In [17]:
prompt = (
    "### Instruction:\n"
    "What is an FIR and what steps should one take to file an FIR in India?\n\n"
    "### Response:"
)

inputs = tokenizer(prompt, return_tensors="pt").to(torch.cuda.current_device())

with torch.no_grad():
    output_ids = model.generate(
        inputs["input_ids"],
        max_length=512,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


### Instruction:
What is an FIR and what steps should one take to file an FIR in India?

### Response: 
An FIR is a First Information Report, a document filed with a police officer detailing an alleged offence. To file an FIR, one should approach the police station and provide details of the offence. The police officer will then investigate the matter. If the offence is cognizable, the police will investigate and file a case. If it's non-cognizable, they may not proceed. Always seek legal advice before proceeding.


In [21]:
prompt = (
    "### Instruction:\n"
    "What is the difference between cognizable and non-cognizable offences under Indian law?\n\n"
    "### Response:"
)

inputs = tokenizer(prompt, return_tensors="pt").to(torch.cuda.current_device())

with torch.no_grad():
    output_ids = model.generate(
        inputs["input_ids"],
        max_length=256,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


### Instruction:
What is the difference between cognizable and non-cognizable offences under Indian law?

### Response: 
Cognizable offences are those that can be tried by a Magistrate without a jury, while non-cognizable offences require a jury trial.  The difference is significant in terms of procedure and evidence.  The distinction is crucial for understanding the rights of the accused.  The difference is significant in terms of procedure and evidence.  The distinction is crucial for understanding the rights of the accused.  The difference is significant in terms of procedure and evidence.  The distinction is crucial for understanding the rights of the accused.  The difference is significant in terms of procedure and evidence.  The distinction is crucial for understanding the rights of the accused.  The difference is significant in terms of procedure and evidence.  The distinction is crucial for understanding the rights of the accused.  The difference is significant in terms of proc

In [23]:
prompt = (
    "### Instruction:\n"
    "Suppose a person is arrested in India without a warrant.\n"
    "What rights does that person have under Indian law?\n\n"
    "Now, assume that this person is my friend, and they were arrested without any prior notice late at night. "
    "What immediate steps can I take to protect their rights or help them?\n\n"
    "If I believe this arrest is based on a false allegation, what legal remedies are available — like applying for bail or anticipatory bail? "
    "What is the process and who should I approach?\n\n"
    "The police are not disclosing where my friend is being held. What can I do legally to find out their location and ensure their safety?\n\n"
    "Finally, if the police fail to produce my friend before a magistrate within 24 hours, what legal consequences do they face and how can I seek justice for this violation?\n\n"
    "### Response:"
)


inputs = tokenizer(prompt, return_tensors="pt").to(torch.cuda.current_device())

with torch.no_grad():
    output_ids = model.generate(
        inputs["input_ids"],
        max_length=256,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


### Instruction:
Suppose a person is arrested in India without a warrant.
What rights does that person have under Indian law?

Now, assume that this person is my friend, and they were arrested without any prior notice late at night. What immediate steps can I take to protect their rights or help them?

If I believe this arrest is based on a false allegation, what legal remedies are available — like applying for bail or anticipatory bail? What is the process and who should I approach?

The police are not disclosing where my friend is being held. What can I do legally to find out their location and ensure their safety?

Finally, if the police fail to produce my friend before a magistrate within 24 hours, what legal consequences do they face and how can I seek justice for this violation?

### Response: 
The arrested person has the right to be produced before a judicial magistrate within 24 hours. If they are not, the police can face legal consequences. I can approach the High Court for a 